In [1]:
pip install --upgrade openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
from dotenv import load_dotenv
import requests
import openai
import json

load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API")
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API")

openai.api_key = OPENAI_API_KEY


In [3]:
def get_channel_id_and_name(api_key, handle ):
    """Fetch the YouTube Channel ID and Name using a @handle (e.g., @ItzNandez)."""
    url = f"https://www.googleapis.com/youtube/v3/channels?part=id,snippet&forHandle={handle}&key={api_key}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if "items" in data and len(data["items"]) > 0:
            return data["items"][0]["id"], data["items"][0]["snippet"]["title"]
    
    print("❌ Error: Could not find channel ID for handle:", handle)
    return None, None

# Probar la función
youtube_handle = "@ItzNandez"  # Reemplázalo con otro handle si lo deseas
channel_id, channel_name = get_channel_id_and_name(GOOGLE_API_KEY, youtube_handle)

if channel_id:
    print(f"✅ Found Channel ID: {channel_id}")
    print(f"📌 Channel Name: {channel_name}")
else:
    print("❌ No se pudo encontrar el canal.")


✅ Found Channel ID: UCwPsQVobGoNteKbzn-tXI_w
📌 Channel Name: Nandez!


In [4]:
import isodate

def is_probable_short(video_id, api_key):
    """Determina si un video es un Short basado en su duración."""
    url = f"https://www.googleapis.com/youtube/v3/videos?key={api_key}&id={video_id}&part=contentDetails"
    response = requests.get(url)
    
    try:
        data = response.json()
        if "items" in data and len(data["items"]) > 0:
            duration_iso = data["items"][0]["contentDetails"]["duration"]
            duration_seconds = int(isodate.parse_duration(duration_iso).total_seconds())

            if duration_seconds < 180:  
                return True
    except Exception as e:
        print(f"❌ Error obteniendo duración del video {video_id}: {e}")

    return False


def get_latest_non_short_videos(api_key, channel_id, max_results=10):
    """Fetch the latest publicly visible videos, filtering out Shorts using duration."""
    url = f"https://www.googleapis.com/youtube/v3/search?key={api_key}&channelId={channel_id}&part=snippet,id&order=date&type=video&maxResults={max_results * 3}"
    response = requests.get(url)

    video_list = []
    try:
        data = response.json()
        if "items" in data:
            for item in data["items"]:
                video_id = item["id"].get("videoId")
                if not video_id:
                    continue

                title = item["snippet"]["title"]
                description = item["snippet"].get("description", "")

                # Filtrar Shorts basados en la duración real
                if is_probable_short(video_id, api_key):
                    continue  

                # Guardar solo videos normales
                video_info = {
                    "videoId": video_id,
                    "title": title,
                    "publishTime": item["snippet"]["publishedAt"]
                }
                video_list.append(video_info)

                if len(video_list) >= max_results:
                    break
    except Exception as e:
        print(f"❌ Error fetching videos: {e}")

    return video_list



In [5]:
videos = get_latest_non_short_videos(api_key=GOOGLE_API_KEY,channel_id='UCwPsQVobGoNteKbzn-tXI_w')
print(videos)
print(len(videos))

[{'videoId': 'bElpDUl-lxc', 'title': 'Dejé Mi Alquiler Para Vivir Por El Mundo', 'publishTime': '2025-02-13T17:11:11Z'}, {'videoId': 'BXD9hAQ7ucw', 'title': 'Pasé 24 Horas Como Un Millonario en Egipto', 'publishTime': '2025-01-01T16:00:11Z'}, {'videoId': 'nN6jfwJ9HIQ', 'title': 'ENCONTRÉ el PARAISO que NADIE CONOCE de EGIPTO - Vlog Día 6', 'publishTime': '2024-12-26T18:00:28Z'}, {'videoId': '000aHR1mbrE', 'title': 'Se nos INCENDIA el TAXI en Egipto🇪🇬 - Vlog Día 5', 'publishTime': '2024-12-22T18:00:00Z'}, {'videoId': 'wTU8PhIT3f8', 'title': 'PROBANDO la PALOMA en EGIPTO - Vlog Día 4', 'publishTime': '2024-12-18T18:00:42Z'}, {'videoId': 'JKCpC7WC_4E', 'title': 'ENCONTRÉ la JOYA ESCONDIDA de EGIPTO', 'publishTime': '2024-12-11T16:00:24Z'}, {'videoId': 'deHggjTLbRk', 'title': 'ESTA es la REALIDAD de VISITAR las PIRAMIDES DE EGIPTO - Vlog Día 2', 'publishTime': '2024-12-05T18:00:33Z'}, {'videoId': 'lzc1QAWzoeQ', 'title': 'ME INTENTAN ROBAR - Vlog EGIPTO Día 1 🇪🇬', 'publishTime': '2024-12-01

In [ ]:
import openai
import json
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()

# Configurar la clave de OpenAI correctamente
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    print("❌ ERROR: La clave OPENAI_API_KEY no está configurada en .env")
else:
    print(f"✅ OPENAI_API_KEY cargada correctamente: {OPENAI_API_KEY[:5]}*****")

# Crear un cliente de OpenAI (nuevo formato)
client = openai.Client(api_key=OPENAI_API_KEY)

def detect_sponsors_openai(description):
    """Usa OpenAI para detectar marcas patrocinadoras en la descripción de un video."""

    if not description:
        print("🔍 No description provided.")
        return []

    prompt = f"""
    You are an expert at extracting **brand names, company mentions, or sponsorships** from a YouTube description.  
    **Only return real companies, brands, or products** that are being advertised, promoted, or mentioned as a sponsor.  
    **DO NOT return:**
    - Personal names (e.g., YouTubers, influencers, content creators).
    - Social media handles, channels, or usernames.
    - Platform names like YouTube, Twitch, Discord, Patreon.

    **Return only real brands or companies** in a JSON **list format** (like ["Brand 1", "Brand 2"]).  
    If no brands are detected, return an empty list **[]**.

    ---
    **Description:** {description}
    """

    print("\n🟢 Enviando solicitud a OpenAI...")
    print(f"🔹 Longitud del prompt: {len(prompt)} caracteres")
    print("🔹 Prompt enviado:\n", prompt[:500])

    try:
        response = client.chat.completions.create( 
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )

        print("\n🔵 Respuesta de OpenAI recibida:\n", response)

        raw_output = response.choices[0].message.content
        print("\n🔵 OpenAI Response (Raw):\n", raw_output)

        # ✅ Intentar convertir la respuesta a JSON
        detected_brands = json.loads(raw_output)

        print("\n🟡 Marcas detectadas:\n", detected_brands)

        return detected_brands if isinstance(detected_brands, list) else []
    
    except json.JSONDecodeError:
        print("❌ ERROR: OpenAI devolvió un formato no válido:\n", raw_output)
        return []
    except Exception as e:
        print(f"❌ ERROR: OpenAI API Error: {e}")
        return []


✅ OPENAI_API_KEY cargada correctamente: sk-pr*****


In [16]:
import requests

def get_video_description(api_key, video_id):
    """Obtiene la descripción de un video de YouTube a partir de su video_id."""
    url = f"https://www.googleapis.com/youtube/v3/videos?key={api_key}&id={video_id}&part=snippet"
    response = requests.get(url)

    try:
        data = response.json()
        if "items" in data and len(data["items"]) > 0:
            description = data["items"][0]["snippet"]["description"]
            return description
    except Exception as e:
        print(f"❌ Error obteniendo la descripción del video: {e}")

    return None  # Devuelve None si hay un error o no se encuentra el video


In [17]:
for video in videos:
    descripcion = get_video_description(api_key=GOOGLE_API_KEY,video_id=video['videoId'])
    detect_sponsors_openai(description=descripcion)


🟢 Enviando solicitud a OpenAI...
🔹 Longitud del prompt: 1062 caracteres
🔹 Prompt enviado:
 
    You are an expert at extracting **brand names, company mentions, or sponsorships** from a YouTube description.  
    **Only return real companies, brands, or products** that are being advertised, promoted, or mentioned as a sponsor.  
    **DO NOT return:**
    - Personal names (e.g., YouTubers, influencers, content creators).
    - Social media handles, channels, or usernames.
    - Platform names like YouTube, Twitch, Discord, Patreon.

    **Return only real brands or companies** in a JS

🔵 Respuesta de OpenAI recibida:
 ChatCompletion(id='chatcmpl-B0x55XsFfNHpsUyt8v2dy76tvOv5w', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='[]\nNo brand names, company mentions, or sponsorships were detected in the YouTube description provided.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1739566807, m